In [290]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm  

In [291]:
print("CUDA Disponible:", torch.cuda.is_available())
print("Nombre de GPU:", torch.cuda.device_count())
print("Nom du GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")
print("Version CUDA:", torch.version.cuda)


CUDA Disponible: True
Nombre de GPU: 1
Nom du GPU: NVIDIA GeForce RTX 2060
Version CUDA: 11.8


In [292]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda


In [293]:
train_df = pd.read_csv("data/train_cleaned.csv")

X = train_df[["brand", "model", "age", "milage", "fuel_type", "engine_hp"]].values
y = train_df["price"].values

In [294]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
X = scaler.fit_transform(X)

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [296]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

In [297]:
print(len(X_train))
print(len(X_test))

113014
28254


In [298]:
print(X.shape, X_train.shape, X_test.shape)

(141268, 6) (113014, 6) (28254, 6)


In [299]:
print(y.shape, y_train.shape, y_test.shape)

(141268,) (113014,) (28254,)


In [300]:
class PricePredictionNN(nn.Module):
    def __init__(self):
        super(PricePredictionNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)
    
model = PricePredictionNN().to(device)

In [301]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [302]:
epochs = 100
batch_size = 64
patience = 10 
best_loss = float('inf') 
early_stop_counter = 0 
best_model_state_dict = None  

In [303]:
for epoch in range(epochs):
    model.train()  # Met le modèle en mode entraînement
    epoch_loss = 0.0
    progress_bar = tqdm(range(0, len(X_train_tensor), batch_size), desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for i in progress_bar:
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())  # Afficher la loss en temps réel

    avg_epoch_loss = epoch_loss / len(progress_bar)  # Loss moyenne pour l'époque

    print(f"Epoch {epoch+1}/{epochs} - Loss moyenne: {avg_epoch_loss:.6f}")

    # Vérification de l'amélioration de la loss (early stopping)
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss  # Mise à jour de la meilleure perte
        early_stop_counter = 0  # Réinitialisation du compteur de patience
        best_model_state_dict = model.state_dict()  # Sauvegarder les poids du meilleur modèle
    else:
        early_stop_counter += 1  # Incrémenter le compteur de patience

    # Si patience est dépassée, arrêter l'entraînement
    if early_stop_counter >= patience:
        print(f"Early stopping déclenché après {epoch+1} époques.")
        break

# Charger les poids du meilleur modèle (le modèle avec la plus petite loss)
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)
    print("Modèle restauré à partir du meilleur état.")

Epoch 1/100 - Loss moyenne: 1227025922.518686


Epoch 2/100 - Loss moyenne: 479059406.024915


Epoch 3/100 - Loss moyenne: 404593281.123443


Epoch 4/100 - Loss moyenne: 346070112.742922


Epoch 5/100 - Loss moyenne: 272573576.557191


...

Epoch 81/100 - Loss moyenne: 161830861.782559


Epoch 82/100 - Loss moyenne: 162070731.904870


Epoch 83/100 - Loss moyenne: 161757833.191393


Epoch 84/100 - Loss moyenne: 162004535.392978


Epoch 85/100 - Loss moyenne: 161917869.445074
Early stopping déclenché après 85 époques.
Modèle restauré à partir du meilleur état.


In [304]:
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train_tensor)
    y_pred_test = model(X_test_tensor)

In [305]:
r2_train = r2_score(y_train, y_pred_train.cpu().numpy())
r2_test = r2_score(y_test, y_pred_test.cpu().numpy())

In [306]:
print(f"R² Train : {r2_train:.4f}")
print(f"R² Test : {r2_test:.4f}")

R² Train : 0.6213
R² Test : 0.6185
